In [9]:
from keras.datasets import mnist
import numpy as np

BATCH_SIEZ = 128
NUM_CLASSES = 10
EPOCH_NUM = 12
IMG_SIZE = (28, 28)

(X_train, y_train), (X_test, y_test) = mnist.load_data()
print X_train.shape, y_train.shape, X_test.shape, y_test.shape
X_train = X_train.reshape(-1, 1, 28, 28).astype(np.float32)
y_train = y_train.astype(np.int32)
X_test = X_test.reshape(-1, 1, 28, 28).astype(np.float32)
y_test = y_test.astype(np.int32)
print X_train.shape, y_train.shape, X_test.shape, y_test.shape

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
(60000, 1, 28, 28) (60000,) (10000, 1, 28, 28) (10000,)


In [10]:
import lasagne
import theano
import theano.tensor as T

# create Theano variables for input and target minibatch
input_var = T.tensor4('X')
target_var = T.ivector('y')

# create a small convolutional neural network
from lasagne.nonlinearities import leaky_rectify, softmax
network = lasagne.layers.InputLayer((None, 1, 28, 28), input_var)
print network.output_shape
network = lasagne.layers.Conv2DLayer(network, 64, (3, 3),
                                     nonlinearity=leaky_rectify)
print network.output_shape
network = lasagne.layers.Conv2DLayer(network, 32, (3, 3),
                                     nonlinearity=leaky_rectify)
print network.output_shape
network = lasagne.layers.Pool2DLayer(network, (3, 3), stride=2, mode='max')
print network.output_shape
network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5),
                                    128, nonlinearity=leaky_rectify,
                                    W=lasagne.init.Orthogonal())
print network.output_shape
network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5),
                                    10, nonlinearity=softmax)
print network.output_shape

# create loss function
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean() + 1e-4 * lasagne.regularization.regularize_network_params(
        network, lasagne.regularization.l2)

# create parameter update expressions
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.001,
                                            momentum=0.9)

# compile training function that updates parameters and returns training loss
train_fn = theano.function([input_var, target_var], loss, updates=updates)

(None, 1, 28, 28)
(None, 64, 26, 26)
(None, 32, 24, 24)
(None, 32, 11, 11)
(None, 128)
(None, 10)


In [11]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

EPOCH_NUM = 100
BATCH_SIZE = 100

from time import time
# train network (assuming you've got some training data in numpy arrays)
start_time = time()
for epoch in range(EPOCH_NUM):
    loss = 0
    for input_batch, target_batch in iterate_minibatches(X_train, y_train, BATCH_SIZE):
        loss += train_fn(input_batch, target_batch)
    print("Epoch %d: Loss %g" % (epoch + 1, loss / len(X_train)))
end_time = time()
print 'Training finished, Time: {}s'.format(end_time - start_time)

Epoch 1: Loss 0.00814042
Epoch 2: Loss 0.00266569
Epoch 3: Loss 0.0020958
Epoch 4: Loss 0.0018363
Epoch 5: Loss 0.0016932
Epoch 6: Loss 0.00159286
Epoch 7: Loss 0.00144866
Epoch 8: Loss 0.00139396
Epoch 9: Loss 0.00129052
Epoch 10: Loss 0.00124638
Epoch 11: Loss 0.00122473
Epoch 12: Loss 0.00118825
Epoch 13: Loss 0.00113873
Epoch 14: Loss 0.00115042
Epoch 15: Loss 0.00107126
Epoch 16: Loss 0.00106649
Epoch 17: Loss 0.0010639
Epoch 18: Loss 0.00102069
Epoch 19: Loss 0.00100904
Epoch 20: Loss 0.000983487
Epoch 21: Loss 0.000977057
Epoch 22: Loss 0.000941218
Epoch 23: Loss 0.000959378
Epoch 24: Loss 0.000919895
Epoch 25: Loss 0.000906229
Epoch 26: Loss 0.00090332
Epoch 27: Loss 0.000882842
Epoch 28: Loss 0.000872779
Epoch 29: Loss 0.000850319
Epoch 30: Loss 0.000855758
Epoch 31: Loss 0.000843646
Epoch 32: Loss 0.000799919
Epoch 33: Loss 0.000827713
Epoch 34: Loss 0.000810285
Epoch 35: Loss 0.000800883
Epoch 36: Loss 0.000805529
Epoch 37: Loss 0.000803478
Epoch 38: Loss 0.000765854
Epoch 3

In [12]:
# use trained network for predictions
test_prediction = lasagne.layers.get_output(network, deterministic=True)
predict_fn = theano.function([input_var], T.argmax(test_prediction, axis=1))
# print("Predicted class for first test input: %r" % predict_fn(X_test))
from sklearn.metrics import accuracy_score
print accuracy_score(y_test, predict_fn(X_test))

0.9936
